In [1]:
import pandas as pd
import os
from PIL import Image
import dlib
import numpy as np
from tensorflow.keras.applications import resnet_v2
import matplotlib.pyplot as plt
from tensorflow import keras
import json
import sys
sys.path.append("./src")
from util import *

In [ ]:
IM_WIDTH = IM_HEIGHT = 224

def rect_to_bb(rect):
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
    return (x, y, w, h)

def detect_face(image_path, default_max_size=800,size = 300, padding = 0.25):
    cnn_face_detector = dlib.cnn_face_detection_model_v1('./dlib_mod/mmod_human_face_detector.dat')
    sp = dlib.shape_predictor('./dlib_mod/shape_predictor_5_face_landmarks.dat')
    base = 2000  # largest width and height

    img = dlib.load_rgb_image(image_path)
    old_height, old_width, _ = img.shape
    old_height, old_width, _ = img.shape

    if old_width > old_height:
        new_width, new_height = default_max_size, int(default_max_size * old_height / old_width)
    else:
        new_width, new_height =  int(default_max_size * old_width / old_height), default_max_size
    img = dlib.resize_image(img, rows=new_height, cols=new_width)
    dets = cnn_face_detector(img, 1)
    num_faces = len(dets)
    if num_faces == 0:
        print("Sorry, there were no faces found in '{}'".format(image_path))
        return
    elif num_faces > 1:
        print("Multiple face in '{}'. A random face will be returned".format(image_path))
    faces = dlib.full_object_detections()
    for detection in dets:
        rect = detection.rect
        faces.append(sp(img, rect))
    image = dlib.get_face_chips(img, faces, size=size, padding = padding)[0]

    image = Image.fromarray(image, 'RGB')
    image = image.resize((IM_WIDTH, IM_HEIGHT))

    #image = np.array(image) / 255.0
    ori_img = np.array(image)
    processed_img = resnet_v2.preprocess_input(np.array(image))
    processed_img = processed_img[None,:]
    return ori_img, processed_img

In [2]:
img_path = "/teams/DSC180A_FA20_A00/a01explainableai/a01capstonegroup03/fairface_pad025/train/2450.jpg"
img_path_2 = "IMG_3533.jpg"#"./test.jpeg"

In [ ]:
a, b = detect_face(img_path_2)

In [4]:
get_prediction(img_path_2, model_path, mapping)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


'East Asian'

In [ ]:
b.shape == (1,224,224,3)

In [ ]:
def get_prediction(img_arr, model_path, mapping_path):
    if img_arr.shape != (1, 224,224,3):
        print("Wrong input size")
        return
    else:
        model = keras.models.load_model(model_path)
        
        with open(mapping_path) as f:
            mapping = json.load(f)
        f.close()
        mapping = {val:key for key, val in mapping.items()}
        pred = model.predict(img_arr)
        out = mapping[pred.argmax()]
    
    return out

In [3]:
np.set_printoptions(suppress=True)
model_path = "./models/race/race_v6.hdf5"
#model = keras.models.load_model(model_path)
mapping = "./mapping/race.json"

In [ ]:
get_prediction(b, model_path, mapping)

In [ ]:
res = model.predict(b)

In [ ]:
res

In [ ]:
with open(mapping) as f:
    mappings = json.load(f)
f.close()

In [ ]:
map_rev = {val:key for key, val in mappings.items()}
map_rev[res.argmax()]

In [ ]:
plt.imshow(b.squeeze())

In [ ]:
plt.imshow(b.squeeze())

In [ ]:
a.squeeze().shape

In [ ]:
plt.imshow(plt.imread(img_path))

In [ ]:
plt.imshow(a.squeeze())